In [2]:
import numpy as np
import itertools

In [3]:
np.set_printoptions(linewidth=200)

In [4]:
def choose_ij(vals, i, j):
    return vals[i] / vals[j]

In [5]:
def choose_both(vals, i, j):
    return choose_ij(vals, i, j) + choose_ij(vals, j, i)

In [6]:
def f_first(vals):
    prod = np.complex128(1)
    for j in range(len(vals)):
        for k in range(j+1, len(vals)):
            prod *= choose_both(vals, j, k)

    return prod

In [7]:
def f_second(vals):
    A = np.zeros([len(vals), len(vals)], dtype=np.complex128)
    for j in range(len(vals)):
        for k in range(len(vals)):
            if j == k:
                continue
            A[j,k] = -1 * choose_ij(vals, j, k) / choose_both(vals, j, k)

    for i in range(len(vals)):
        # check if this is row or col sum
        A[i, i] = -1 * A[i, :].sum()

    return np.linalg.det(A[:-1, :-1])

In [8]:
def f(vals):
    return f_first(vals) * f_second(vals)

In [9]:
def count_cycles(n, m):
    roots = np.roots([1] + [0]*(m-1) + [-1])

    s = np.complex128()

    for x in itertools.product(roots, repeat=n-1):
        s += f(x + (np.complex128(1),))
    
    return s / m ** (n-1)

In [13]:
# So - our calculation has given us both the constant term - and also the terms where the exponents are all in {0, -6, 6}
# The exponents of the x_i are calculating the "net out degree". So note that the only vertex that can have net out degree is the root vertex of our tree - v_n
# Note also that the total degree of each term is zero. So therefore these terms must all be of the form x_i^6*x_n*-6 - i!= n
# We could try and calculate this term directly (and multiply by n-1 for the n-1 choices for i at the end)
# Wlog take i = n-1
# We want to force our graph to have directed edges E_n-1,i and E_i,n. 
# So we can take an adjusted f_first with all these coefficients
# Then in our matrix we could fill things as normal but put a zero if E_i,n-1 or E_n,i is in the tree, and put a 1 where E_n-1,i or E_i,n in the tree
# This also lets us factor x_i, x_n out of our formula. Taking these two out both adds and removes 1 from each degree putting us back where we started

In [48]:
def f_second_mod(vals):
    n = len(vals)+2
    A = np.zeros([n, n], dtype=np.complex128)
    for j in range(len(vals)):
        for k in range(len(vals)):
            if j == k:
                continue
            A[j,k] = -1 * choose_ij(vals, j, k) / choose_both(vals, j, k)

    for i in range(n):
        # if i != n-1:
        A[i, n-1] = -1
        A[n-1, i] = 0

        if i != n-2:
            A[i, n-2] = 0
            A[n-2, i] = -1

        # There's an extra minus one here???
        
        

    for i in range(n):
        # check if this is row or col sum
        A[i, i] = -1 * A[i, :].sum()

    # return A
    
    return np.linalg.det(A[:-1, :-1])

In [49]:
# f_second_mod([1,1,1,1,1])

In [50]:
def jack_offset_f(vals):
    return f_first(vals) * f_second_mod(vals)

In [54]:
def calc_jack_offset(n, m):
    roots = np.roots([1] + [0]*(m-1) + [-1])
    
    s = np.complex128()
    
    for x in itertools.product(roots, repeat=n-3):
        s += jack_offset_f(x + (np.complex128(1),))
    
    return (n-1) * s / m ** (n-3)

In [55]:
count_cycles(7, 5)

np.complex128(1015439.999999994-3.883838653564453e-10j)

In [56]:
calc_jack_offset(7, 3)

np.complex128(156383.9999999999+0j)

In [57]:
count_cycles(7, 3) - calc_jack_offset(7, 3)

np.complex128(1015439.999999999-3.768726467922718e-11j)

In [58]:
26064 * 6

156384

In [318]:
calc_jack_offset(11, 5)

np.complex128(357454162239901.4+0.0069236363636363644j)

In [332]:
357454162240000 + 169107043478365440 - 169464497640605440

0

In [265]:
39319957846389 / 1117044257 / 2**7 / 5 ** 2

10.999999999996923

In [221]:
apprx_cycles_11 = count_cycles(11, 5)
apprx_cycles_11

KeyboardInterrupt: 

In [ ]:
count_cycles(7, 3)

In [172]:
m=6
n=7
roots = np.roots([1] + [0]*(m-1) + [-1])

s = np.complex128()

for x in itertools.product(roots, repeat=n-3):
    s += jack_offset_f(x + (np.complex128(1),))

(n-1)*s / m ** (n-3)

np.complex128(11088-1.6842494477276448e-14j)

In [157]:
11088 / 2**4 / 3**2

77.0

In [227]:
((39319957846389 / 11) + 1) * 10 * 10 

357454162240000.0

In [226]:
357454162240000

357454162240000

In [ ]:
# def f_fst_mod(vals):
#     prod = np.complex128(1)
#     for i in range(1, len(vals)):
#         prod *= choose_ij(vals, 0, i)
    
#     for j in range(1, len(vals)):
#         for k in range(j+1, len(vals)):
#             prod *= choose_both(vals, j, k)

    

#     return prod

In [336]:
(14 * 14 * 169464497640605440 // 15)

2214336102503911082

In [337]:
4435711276305905572695127676467200 - 4435791580898238776281705120204800 + (14 * 14 * 169464497640605440 // 15)

-80304592330989250474939826518

In [339]:
(4435791580898238776281705120204800 - 4435711276305905572695127676467200 ) / 169464497640605440

473872660358.10547